request와 BeautifulSoup으로 웹크롤러 만들기
==

request와 BeautifulSoup을 통해서 뉴욕타임즈의 세계 뉴스 항목을 가져와 보겠습니다.

In [1]:
# parser.py
import requests

# HTTP GET Request
req = requests.get('https://namu.wiki/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%8C%80%EB%AC%B8')

# HTML 소스 가져오기
html = req.text
# HTTP Header 가져오기
header = req.headers
# HTTP Status 가져오기 (200: 정상)
status = req.status_code
# HTTP가 정상적으로 되었는지 (True/False)
is_ok=req.ok

Requests 라이브러리로 HTML을 가져올 수는 있지만 이것만으로 html을 파이썬이 이해하는 객체 구조로 만들어주지는 않습니다. 또한 req.text는 파이썬의 문자열(str) 객체를 변환할 뿐이기에 정보를 추출하기는 어렵습니다.

그러므로 BeautifulSoup를 이용하는 것이며, BS는 html 코드를 파이썬이 이해하는 객체구조로 변환하는 "Parsing"이라는 작업을 해주므로, "의미있는" 정보추출에 효과적입니다.

고로 밑은 BS를 이용하여 Parser.py를 좀더 가다듬어 보겠습니다.

In [2]:
# parser.py
import requests
from bs4 import BeautifulSoup

# HTTP GET Request
req = requests.get('https://www.nytimes.com/section/world')

# HTML 소스 가져오기
html = req.text
# BeautifulSoup으로 html 소스를 파이썬 객체로 변환합니다.
#첫 인자는 html 소스코드이며, 두번째 인자는 어느 parser를 이용하는지 명시합니다.
#여기서는 파이썬 내장 html.parser를 이용해봅시다.
soup = BeautifulSoup(html, 'html.parser')

이제 soup 객체에서 원하는 정보를 찾아낼 수 있게 되었습니다.

BeautifulSoup는 여러가지 기능을 제공하는데 그중 하나는 select입니다.
select는 CSS Selector를 이용해 조건에 일치하는 객체들을 List로 반환해 줍니다.

https://www.nytimes.com/section/world

위 주소에서 뉴스 기사 제목인 "Within Venezuelan Military Ranks, a Struggle Over What Leader to Back"에 오른쪽 클릭을 한후 검사(inspect)를 누르면 오른쪽에 검사도구가 나오는것을 알 수 있습니다.

원하는 부분을 오른쪽 클릭 > Copy > Copy Selecter를 통해 복사하고 붙여 넣어보면 다음과 같이 나오는걸 알 수 있습니다.

#collection-world > div.rank > section.highlights.highlights-collection-package.collection-package.collection > div.rank-template.featured-rank-template.template-2 > ol > li:nth-child(1) > article > div > h2 > a

하지만 :nth-child(1) 등이 붙어있는 것으로 보아 현재 요소를 '정확하게' 특정하고 있기 때문에, 좀 더 유연하게 만들어 주기 위해 아래와 같이 selector를 바꿔줍니다.(위 코드는 단 하나의 링크만을 특정하고, 아래 코드는 css selector에 일치하는 모든 요소를 가리킵니다.)

h2 > a

In [3]:
## parser.py
import requests
from bs4 import BeautifulSoup

req = requests.get('https://www.nytimes.com/section/world')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
## CSS Selector를 통해 html요소들을 찾아낸다.
my_titles = soup.select(
    'h2 > a'
    )

In [10]:
## 이제 뉴스기사 제목을 가져와 봅시다.

## parser.py
import requests
from bs4 import BeautifulSoup

req = requests.get('https://www.nytimes.com/section/world')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
my_titles = soup.select(
    'h2 > a'
    )
## my_titles는 list 객체
for title in my_titles:
    ## Tag안의 텍스트
    print(title.text)


The New York Times

Within Venezuelan Military Ranks, a Struggle Over What Leader to Back
Mexico Moves to Encourage Caravan Migrants to Stay and Work
7 People Killed and 200 Missing in Brazil After Dam Collapses, Officials Say
New Taliban Negotiator Seen as Sign That This Time, Afghan Peace Talks Are Serious
Did the Queen Just Weigh In on Brexit?
Arrests in Terror Plot Raise Questions About Canada’s Refugee Embrace
Venezuela’s Opposition Leader Calls for More Protests ‘if They Dare to Kidnap Me’
U.N. Execution Expert Will Investigate Khashoggi Killing
Greeks Approve Deal to Rename Macedonia, in Victory for the West
Huge Trove of Leaked Russian Documents Is Published by Transparency Advocates
Mexico Protests U.S. Decision to Return Asylum Seekers
‘Individual Plaintiff’ Brought Case Against Imprisoned Navy Veteran, Iran Says
Across Australia, Yet Another Scorching Summer
This Old Basque Soldier Expected to Die 80 Years Ago
Dispatches
Cambodia Ruler’s Recipe for Youth Appeal? An 8,900-Po

In [11]:
#응용으로 뉴스기사 제목과 해당 제목의 주소를 가져와 봅시다.

## parser.py
import requests
from bs4 import BeautifulSoup

req = requests.get('https://www.nytimes.com/section/world')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
my_titles = soup.select(
    'h2 > a'
    )
## my_titles는 list 객체
for title in my_titles:
    ## Tag안의 텍스트
    print(title.text)
    ## Tag의 속성을 가져오기(ex: href속성)
    print(title.get('href')) ##주소부분


The New York Times

https://www.nytimes.com/
Within Venezuelan Military Ranks, a Struggle Over What Leader to Back
https://www.nytimes.com/2019/01/25/world/americas/venezuela-military-maduro.html
Mexico Moves to Encourage Caravan Migrants to Stay and Work
https://www.nytimes.com/2019/01/25/world/americas/migrant-caravan-honduras-mexico.html
7 People Killed and 200 Missing in Brazil After Dam Collapses, Officials Say
https://www.nytimes.com/2019/01/25/world/americas/brazil-dam-burst-brumadinho.html
New Taliban Negotiator Seen as Sign That This Time, Afghan Peace Talks Are Serious
https://www.nytimes.com/2019/01/25/world/asia/taliban-negotiator-afghanistan.html
Did the Queen Just Weigh In on Brexit?
https://www.nytimes.com/2019/01/25/world/europe/queen-elizabeth-brexit-britain.html
Arrests in Terror Plot Raise Questions About Canada’s Refugee Embrace
https://www.nytimes.com/2019/01/25/world/canada/terror-plot-arrests-refugees.html
Venezuela’s Opposition Leader Calls for More Protests ‘i